In [1]:
from __future__ import division
import numpy as np

import posterior_agreement

from matplotlib import pyplot as plt
%matplotlib inline

# Create fake "MCMC chains"

In [2]:
# Create Npoints samples from random multivariate, nDim-dimensional Gaussian
def create_random_samples(nDim, Npoints):
    means = np.random.rand(nDim)
    cov = .5 - np.random.rand(nDim**2).reshape((nDim,nDim))
    cov = np.triu(cov)
    cov += cov.T - np.diag(cov.diagonal())
    cov = np.dot(cov,cov)
    samples =  np.random.multivariate_normal(means, cov, Npoints)
    return samples

# Create two sets of fake data with 3 parameters
np.random.seed(42) # To be able to create the same fake data over and over again
samples1 = create_random_samples(3, 50000)
samples2 = 1+create_random_samples(3, 70000)

## One-dimensional parameter agreement
Let's find out how well both data sets agree in their first parameter!

In [3]:
chains = (samples1[:,0], samples2[:,0])
agreement_1d = posterior_agreement.compute_agreement(chains)

In [4]:
print "p-value %.3f, corresponding to %.1f sigmas"%(agreement_1d[0], agreement_1d[1])

p-value 0.410, corresponding to 0.8 sigmas


## Two-dimensional parameter agreement
Now let's see what the agreement is in the space of parameters one and three. Note that this may take a little while to run. If too impatient, try and play with the `nSamples` and `nBins` arguments.

In [5]:
chains = (samples1[:,[0,2]], samples2[:,[0,2]])
agreement_2d = posterior_agreement.compute_agreement(chains)

In [6]:
print "p-value %.3f, corresponding to %.1f sigmas"%(agreement_1d[0], agreement_1d[1])

p-value 0.410, corresponding to 0.8 sigmas


## Word of Warning
How you compare your MCMC chains should really depend on the exact question you are trying to answer. The above examples do not tell you how well both data sets agree (as you would have to look at all parameters!). The above examples really tell you how well the data sets agree in that one parameter, or these two parameters.

# Notes about what the number of sigmas means

## 1D analytic example
Let's draw from a one-dimensional Gaussian distribution $\mathcal N(0,1)$, and from a $\delta$ function centered at 1 or 2. We know the answer: this should give $1\sigma$ and $2\sigma$ difference. And it does!

In [7]:
samples0 = np.random.randn(10000)
samples1 = np.ones(10)
samples2 = 2*np.ones(10)

In [8]:
agreement_1d_1 = posterior_agreement.compute_agreement((samples0, samples1), nSamples=50000, nBins=60)
agreement_1d_2 = posterior_agreement.compute_agreement((samples0, samples2), nSamples=50000, nBins=60)

In [9]:
print "Example one: %.3fσ, exact solution is 1"%agreement_1d_1[1]
print "Example two: %.3fσ, exact solution is 2"%agreement_1d_2[1]

Example one: 0.970σ, exact solution is 1
Example two: 2.000σ, exact solution is 2


## 2D analytic example
Same exercise, but in two dimensions. Naively, you might think we should get $1\sigma$ and $2\sigma$ differences. But we don't. So what's going on? Well, remember we are asking what the probability to exceed zero difference is, and we convert this $p$-value to sigmas. We are not asking whether 0 difference coincides with the $1\sigma$ or $2\sigma$ isocontour.

In [10]:
cov = [[1,0], [0,1]]
samples0 =  np.random.multivariate_normal([0,0], cov, 50000)
samples1 = np.concatenate((np.ones(10)[:,None],np.zeros(10)[:,None]), axis=1)

In [11]:
agreement_2d_1 = posterior_agreement.compute_agreement((samples0, samples1), nSamples=50000, nBins=60)

In [12]:
print "Example one: PTE %.3f, %.3fσ"%(agreement_2d_1[0], agreement_2d_1[1])

Example one: PTE 0.615, 0.503σ
